# File Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import os
import json
import datetime
import pysmile
import pysmile_license
import sys
import json
from colormap import rgb2hex
from collections import ChainMap
import re
from scipy.stats import lognorm
import scipy
import math
from scipy import stats
from IPython import embed
import geopandas as gpd
import folium
import shapely
from shapely.geometry import LineString, shape
from folium.plugins import FloatImage
import itertools
import pyproj
import ipywidgets

# Transforming from m to degrees
transformer = \
    pyproj.Transformer.from_crs(pyproj.CRS("EPSG:32760"),pyproj.CRS("EPSG:4326")) 

# Load and format data, network

In [2]:
with open('Processed_data_and_output/Autogen_clusters/number_of_clusts.txt') as f:
    cluster_count = int(f.readlines()[0])

In [3]:
# This is duplicated from BN notebook. Need to remove this
def data_gen(clust,shoreline_nodes,binary_nodes,node_exceptions,nodes_to_consider,df_shoreline,net):
    df_shoreline = df_shoreline[df_shoreline.proxy=='TOB']
    
    df_shoreline.index = df_shoreline.year2

    df_shoreline = df_shoreline.rename(
        columns={
            'intersect_distance':'TOB',
            'previous_mean_shoreline_change':'Previous',
            'other_locs':'Surroundings',
            'shoreline_direction':'ShorelineDirection',
            'avg_slope_change':'ShorelineCurvature',
            'facing_north?':'Facingnorth',
            'facing_east?':'Facingeast',
            'wave_energy_mid':'WavesPerpendicular',
            'wave_energy_right':'WavesRight',
            'wave_energy_left':'WavesLeft',
        })
    
    [df_shoreline.rename(columns={x:x.capitalize()},inplace=True) for x in list(df_shoreline) if x not in ['lat','lon','lat_avg','lon_avg','x','y','lat_new','lon_new']]
    
    #normalise the shoreline change variables
    for var in shoreline_nodes:
        val_list = []
        for index,row in df_shoreline.iterrows():
            if row[var]<-3:
                val = 'Erosion'
            elif row[var]>3:
                val = 'Accretion'
            else:
                val = 'Stable'
            val_list.append(val)
        
        df_shoreline[var] = val_list

        
    df_shoreline = df_shoreline[df_shoreline.Mei<=0.5] # removing el nino since there's not enough data
    df_shoreline['Mei'] = ['Neutral' if (x>=-0.5) else 'LaNina' for x in df_shoreline.Mei]
    
    net_nodes = net.get_all_node_ids()
    net_nodes = list(set([re.sub(r'\d+','',x) for x in net_nodes]))
    
    for var in list(df_shoreline[net_nodes]):
        if var not in shoreline_nodes+node_exceptions+binary_nodes:
            df_shoreline[var] = ['Low' if (x<np.mean(df_shoreline[var])) else 'High' for x in df_shoreline[var]]
            
    for var in list(df_shoreline[net_nodes]):
        if type(df_shoreline.reset_index().loc[0,var])==np.bool_:
            df_shoreline[var] = df_shoreline[var].astype(str)

    return(df_shoreline)

In [4]:
shoreline_list = []

for clust in range(cluster_count):
    df_shoreline = pd.read_csv(f'Processed_data_and_output/Autogen_clusters/clust_{clust}.csv')

    df_shoreline['lat_avg'] = [transformer.transform(x,y)[0] for x,y in zip(df_shoreline['x_avg'],df_shoreline['y_avg'])]
    df_shoreline['lon_avg'] = [transformer.transform(x,y)[1] for x,y in zip(df_shoreline['x_avg'],df_shoreline['y_avg'])]

    df_shoreline['lat_new'] = [transformer.transform(x,y)[0] for x,y in zip(df_shoreline['x_new'],df_shoreline['y_new'])]
    df_shoreline['lon_new'] = [transformer.transform(x,y)[1] for x,y in zip(df_shoreline['x_new'],df_shoreline['y_new'])]

    # Pick a random year to work with
    df_shoreline = df_shoreline[df_shoreline.year2==2005]
    
    shoreline_list+=[df_shoreline]

df_shoreline_combined = pd.concat(shoreline_list)

In [5]:
all_clusters_dict = {}
net_dict = {}

for clust in range(cluster_count):
    df_shoreline = pd.read_csv(f'Processed_data_and_output/Autogen_clusters/clust_{clust}.csv')

    df_shoreline['lat_avg'] = [transformer.transform(x,y)[0] for x,y in zip(df_shoreline['x_avg'],df_shoreline['y_avg'])]
    df_shoreline['lon_avg'] = [transformer.transform(x,y)[1] for x,y in zip(df_shoreline['x_avg'],df_shoreline['y_avg'])]

    df_shoreline['lat_new'] = [transformer.transform(x,y)[0] for x,y in zip(df_shoreline['x_new'],df_shoreline['y_new'])]
    df_shoreline['lon_new'] = [transformer.transform(x,y)[1] for x,y in zip(df_shoreline['x_new'],df_shoreline['y_new'])]

    # Pick a random year to work with
    df_shoreline = df_shoreline[df_shoreline.year2==2005]

    shoreline_nodes = ['Tob','Previous']# 'Surroundings','Previous'
    binary_nodes = ['Tcseason']
    node_exceptions = ['Mei','Season']
    nodes_to_consider = ['TOB','MEI','TCSeason','ShorelineCurvature','season','Facingnorth','Facingeast','Previous',
                    'WavesPerpendicular','WavesRight','WavesLeft']     
    net = pysmile.Network()
    net.read_file(f'Processed_data_and_output/reversed_arcs_combined.xdsl')

    df_shoreline = data_gen(clust,shoreline_nodes,binary_nodes,node_exceptions,nodes_to_consider,df_shoreline,net)
    
    all_clusters_dict.update({
        clust:df_shoreline
    })
    


KeyError: 0

# Functions

In [ ]:
net = pysmile.Network()
net.read_file(f'Processed_data_and_output/reversed_arcs_combined.xdsl')

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse(f'Processed_data_and_output/reversed_arcs_combined.xdsl')
root = tree.getroot()

In [ ]:
for child in root:
    print(child.tag,child.attrib)

In [ ]:
evidence_dict = {}

for case in root[1]:
    if case.attrib['name']=='testcase':
        for node_case in case:
            evidence_dict.update({
                node_case.attrib['node']:node_case.attrib['state']
            })
                

In [ ]:
target_node = 'Tob'

In [ ]:
def shoreline_map(net,target_node):
    
    view = 'Satellite'
    location = [-5.667723, 176.094928]
    if view == 'Map':
        map_osm = folium.Map(location=location,zoom_start=11)
    elif view == 'Satellite':
        token = "pk.eyJ1Ijoic2hhbm5vbi1iZW5ndHNvbiIsImEiOiJja3F1Y2Q0dHEwMzYwMm9wYmtzYzk2bDZuIn0.5jGMyEiJdmXs1HL7x3ThPw" # your mapbox token
        tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

        map_osm = folium.Map(location=location, zoom_start=40, tiles=tileurl, attr='Mapbox')

    output_list = []
    
    net.clear_all_evidence()

    for clust in range(cluster_count):
        
        node_id = target_node+str(clust)
        
        # Update network evidence and get beliefs
        
        
        evidence_dict_subset = {key:value for key,value in evidence_dict.items() if re.sub('\D', '',key)==str(clust)}

        for node,value in evidence_dict_subset.items():

            children = net.get_child_ids(node)
            node_id = net.get_node(node)

            for child in children:
                child_outcomes = net.get_outcome_ids(child)
                child_evidence_dict = {}
                for child_outcome in child_outcomes:

                    child_evidence_dict.update({
                        child_outcome:[1 if row[child]==child_outcome else 0][0]
                    })

                evidence_list = [child_evidence_dict[x] for x in child_outcomes]
                net.set_virtual_evidence(child,evidence_list)

    net.update_beliefs()
    
    for clust,group in df_shoreline_combined.groupby(f'clust_{clust}'):
        feature_gp1 = folium.FeatureGroup(name=f"Cluster {clust}")
        for index,row in group.iterrows():
            folium.CircleMarker(location=[row.lat_avg,row.lon_avg],radius=1,weight=5).add_to(feature_gp1)
                                    # popup=pd.DataFrame(row[['lat','lon','Tcseason','Mei','Spring','Winter','Autum','Summer','Tob']])).add_to(feature_gp1)
        feature_gp1.add_to(map_osm)    
       
    
    for index,row in df_shoreline_combined.iterrows():
        
        
         
        clust = row[f'clust_{cluster_count}']
        node_id = target_node+str(clust)
        
        probs_model = {outcome:val for outcome,val in zip(net.get_outcome_ids(node_id),net.get_node_value(node_id))}

        most_prob_state = max(probs_model,key=probs_model.get)

        opacity = (probs_model[most_prob_state]-0.34)/(1-0.34)

        if most_prob_state=='Erosion':
                folium.PolyLine(locations=[[row.lat_avg,row.lon_avg],
                               [row.lat_avg-(row.lat_new-row.lat_avg)/10,row.lon_avg-(row.lon_new-row.lon_avg)/10]],
                                color='red',opacity=opacity).add_to(map_osm)
        elif most_prob_state=='Accretion':
                folium.PolyLine(locations=[[row.lat_avg,row.lon_avg],
                               [row.lat_avg+(row.lat_new-row.lat_avg)/10,row.lon_avg+(row.lon_new-row.lon_avg)/10]],
                                color='blue',opacity=opacity).add_to(map_osm)
        elif most_prob_state=='Stable':
                folium.PolyLine(locations=[[row.lat_avg-(row.lat_new-row.lat_avg)/40,row.lon_avg-(row.lon_new-row.lon_avg)/40],
                               [row.lat_avg+(row.lat_new-row.lat_avg)/40,row.lon_avg+(row.lon_new-row.lon_avg)/40]],
                                color='white',opacity=opacity).add_to(map_osm)
                    
    
    folium.LayerControl().add_to(map_osm)

    return(map_osm)

# Visualise

In [ ]:
shoreline_map(net,target_node)

In [ ]:
w = ipywidgets.Dropdown(
options=['LaNina','Mean'],
value='LaNina',
description='ENSO',
disabled=False)

In [ ]:
# Update network evidence and get beliefs
net.clear_all_evidence()

for node,value in evidence_dict.items():
    
    children = net.get_child_ids(node)
    node_id = net.get_node(node)
    
    for child in children:
        child_outcomes = net.get_outcome_ids(child)
        child_evidence_dict = {}
        for child_outcome in child_outcomes:

            child_evidence_dict.update({
                child_outcome:[1 if row[child]==child_outcome else 0][0]
            })

        evidence_list = [child_evidence_dict[x] for x in child_outcomes]
        net.set_virtual_evidence(child,evidence_list)

net.update_beliefs()

In [ ]:
probs_model = {outcome:val for outcome,val in zip(net.get_outcome_ids(node_id),net.get_node_value(node_id))}

In [ ]:

probs_model